In [1]:
# This section can be skipped if these libraries are already installed
# !pip install textdescriptives
# !pip install contractions
# !pip install stanza

In [2]:
import numpy as np
import pandas as pd
import re
import csv
import spacy
import textdescriptives as td
from nltk.stem import PorterStemmer
import contractions # TODO: apply contractions
import stanza

nlp = spacy.load("en_core_web_sm")

# TODO: MAKE THIS LESS REDUNDANT

/Users/emicatx/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# ***NEED TO EXPLAIN THIS (LOADING FILES)***

In [3]:
processed_transcripts = pd.read_csv('processed_data/transcripts_spacy_formatted.csv').drop(['Unnamed: 0'],axis = 1)
display(processed_transcripts)

,Text,Coherence
0,"3,000 years have I been fighting. Every mornin...",3
1,"“Dad, you're on TV again !” I heard Eric's voi...",4
2,"When Tyler entered the ward, his daughter Vale...",4
3,His body was failing. He had taken care of it ...,4
4,"I saw the button. It was simple, red, no words...",5
...,...,...
1534,The human body is an amazing thing. It is able...,3
1535,"Once upon a time, a young woman broke her vase...",4
1536,"Once upon a time, two beautiful diamonds and a...",3
1537,Lora and her team had been searching for the l...,5


In [4]:
story_lists = []
with open('processed_data/sentences.csv', 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        story_lists.append(row)
print(story_lists[1000])

["'- ([ REDACTED ] The guy who blazed the trail walked up to Samantha at the door with a massive harpoon in his hand", ' Samantha knelt down beside him and he quickly told her he could not let the rabbit die, a wizard of pure magicks could bring such incredible firepower to a small world', " '- [ BLANKED ] /u/davidvoracious entered his shadow home", " 'There was once a country'that rose before history actually began", " 'where he saw the once dominant empire crumble into dust over 25th century shards of world upon world destroyed by his hammer over 2500 years ago", " 'He is no longer the ruler of his empire though, his other great trump card has now returned to reclaim dominion", " '", ' Beyond the edge of the earth there was a new territory, there still is', " '- ( REDACTED ) Several weeks have passed since The Man from Outer Space landed", ' We are standing on the brink of an interstellar war, we can take action, but first one must be done', ' had master decided to do so so, his lone

In [5]:
feature_set = pd.DataFrame()

# ***NEED TO EXPLAIN THIS (COHESION)***

***also, add examples?***

In [ ]:
# Cohesion (noun overlap)

# TODO: consider whether this should be only sentences rather than utterances, as in story_lists

noun_pos = ['NOUN', 'PROPN', 'PRON']

story_lists_updated = []
for i in range(len(story_lists)):
    story = story_lists[i]
    story_nouns = []
    for j in range(len(story)):
        sentence_nouns = []
        sentence = str(story[j])
        doc = nlp(contractions.fix(sentence.lower()))
        for token in doc:
            if token.pos_ in noun_pos:
                sentence_nouns.append(str(token))
        story_nouns.append(sentence_nouns)
    story_lists_updated.append(story_nouns)

all_cohesion_arrays = np.ndarray((len(story_lists),), dtype = np.ndarray)

for k in range(len(story_lists_updated)):
    string = story_lists_updated[k]
    cohesion_array = np.zeros((len(string), len(string)))
    for i in range(len(string)):
        current_string = string[i]
        for j in range(len(string)):
            new_string = string[j]
            if any(item in new_string for item in current_string):
                cohesion_array[i, j] = 1
    all_cohesion_arrays[k] = cohesion_array

for array in all_cohesion_arrays: # TODO: describe check for symmetry
    if not np.array_equal(array, array.T):
        raise ValueError("Array must be symmetric; check input")


In [ ]:
# Co-reference cohesion, local

# TODO: consider only one sentence long transcripts?

for n, array in enumerate(all_cohesion_arrays):
    counter = 0
    for i in range(array.shape[0] - 1):
            counter += array[i, i+1]
    try:
        coref_local = counter / (array.shape[0] - 1)
    except ZeroDivisionError:
        coref_local = 0 

    feature_set.loc[n, 'coref_local'] = coref_local

feature_set = feature_set.sort_index()

In [ ]:
# Co-reference cohesion, global

np.seterr('raise')

for i, array in enumerate(all_cohesion_arrays):
    n = array.shape[0]
    counter = (array.sum() - np.diag(array).sum()) / 2
    try:
        coref_global = counter / (n * (n - 1) / 2)
    except Exception as e:
        coref_global = 0 

    feature_set.loc[i, 'coref_global'] = coref_global

# ***NEED TO EXPLAIN THIS (TEXT DESCRIPTION)***

***move this down to the bottom!!***

In [ ]:
text_description = pd.DataFrame()

for i in processed_transcripts.index:  
    string = processed_transcripts['Text'].loc[i]
    doc = nlp(string)
    text_description_df = td.extract_metrics(text = string, metrics = ["readability", "dependency_distance", "coherence"], spacy_model = 'en_core_web_sm')
    text_description = pd.concat([text_description_df, text_description])

text_description_ordered = (text_description[::-1]).reset_index()
text_description_ordered = text_description_ordered[['text', 'dependency_distance_mean', 'prop_adjacent_dependency_relation_mean', 'first_order_coherence', 'second_order_coherence']]
display(text_description_ordered)

/Users/emicatx/Library/Python/3.9/lib/python/site-packages/textdescriptives/components/coherence.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append(sent.similarity(sents[i + order]))


In [ ]:
text_description_ordered.to_csv('text_description_ordered.csv')

# ***NEED TO EXPLAIN THIS (STANZA NP/VP/SYNTAX)***

In [ ]:
stanza.download('en')
model = stanza.Pipeline(lang='en', processors='tokenize, pos, constituency')

2025-12-02 00:18:18 INFO: Downloaded file to /Users/emicatx/stanza_resources/resources.json
2025-12-02 00:18:18 INFO: Downloading default packages for language: en (English) ...
2025-12-02 00:18:19 INFO: File exists: /Users/emicatx/stanza_resources/en/default.zip
2025-12-02 00:18:21 INFO: Finished downloading models and saved to /Users/emicatx/stanza_resources
2025-12-02 00:18:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-12-02 00:18:21 INFO: Downloaded file to /Users/emicatx/stanza_resources/resources.json
2025-12-02 00:18:21 WARNING: Language en package default expects mwt, which has been added
2025-12-02 00:18:22 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2025-12-02 00:18:22 INFO: Using device: cpu
2025-12-02 00:18:22 INFO: Loading: tokenize
2025-12-02 00:18:22 INFO: Loading: mwt
2025-12-02 00:18:22 INFO: Loading: pos
2025-12-02 00:18:24 INFO: Loading: constituency
2025-12-02 00:18:24 INFO: Done loading processors!


In [ ]:
# Segmentation

processed_transcripts_updated = []

for i in processed_transcripts.index:
    string = processed_transcripts['Text'].loc[i]
    doc = model(string)
    story_updated = []
    for sentence in doc.sentences:
        tree = sentence.constituency
        string_tree = str(tree).split(' ')
        story_updated.append(string_tree)

    processed_transcripts_updated.append(story_updated)

In [ ]:
processed_transcripts_updated

with open("processed_data/parsed_data.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(processed_transcripts_updated)

In [ ]:
# Higher-level constituent counter

counter_pos = ["(NP", "(VP", "(SBAR", "(PP"]

for i, story in enumerate(processed_transcripts_updated):
    counter = 0
    type_counter = [0, 0, 0, 0]
    for sentence in story:
        for item in sentence:
            if item in counter_pos:
                counter += 1
                idx = counter_pos.index(item)
                type_counter[idx] += 1

    feature_set.loc[i, 'n_constituents'] = counter
    feature_set.loc[i, 'n_noun_phrases'] = type_counter[0]
    feature_set.loc[i, 'n_verb_phrases'] = type_counter[1]
    feature_set.loc[i, 'n_sub_clauses'] = type_counter[2]
    feature_set.loc[i, 'n_prep_phrases'] = type_counter[3]

display(feature_set)

,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases
0,0.550000,0.433333,152.0,79.0,49.0,11.0,13.0
1,0.650000,0.638095,196.0,93.0,68.0,12.0,23.0
2,0.148148,0.096959,599.0,296.0,183.0,41.0,79.0
3,0.152941,0.105062,546.0,285.0,172.0,17.0,72.0
4,0.600000,0.545455,123.0,64.0,40.0,11.0,8.0
...,...,...,...,...,...,...,...
1534,0.461538,0.384615,153.0,70.0,55.0,14.0,14.0
1535,0.600000,0.400000,64.0,27.0,27.0,5.0,5.0
1536,0.166667,0.285714,70.0,33.0,20.0,5.0,12.0
1537,0.500000,0.600000,97.0,49.0,36.0,3.0,9.0


In [ ]:
# Noun and verb phrase parser

noun_phrases_corpus = []
verb_phrases_corpus = []
closing_pattern = r'\)'

phrasal_pos = ["(NP", "(VP"]

for story in processed_transcripts_updated:
    noun_phrases_overall = []
    verb_phrases_overall = []
    for sentence in story:
        noun_phrases = []
        verb_phrases = []
        counter = 0
        for i in range(len(sentence)):
            item = sentence[i]
            if item in phrasal_pos:
                open_counter = 1
                for j in range(i + 1, len(sentence)):
                    first_element = sentence[j][0]
                    match = re.findall(closing_pattern, sentence[j])
                    closed = len(match)
                    if first_element == '(':
                        open_counter += 1
                    open_counter -= closed
                    if open_counter <= 0:
                        break
                phrase = [item for item in sentence[i:j + 1] if item[-1] == ')']
                if item == counter_pos[0]:
                    noun_phrases.append(phrase)
                else:
                    verb_phrases.append(phrase)

        noun_phrases_overall.append(noun_phrases)
        verb_phrases_overall.append(verb_phrases)

    noun_phrases_corpus.append(noun_phrases_overall)
    verb_phrases_corpus.append(verb_phrases_overall)

In [ ]:
# Mean phrase lengths

def phrase_length(phrase_corpus: list, phrase_lengths: list) -> list:
    
    # TODO: type hints + docstrings

    for story in phrase_corpus:
        length_list = []
        for sentence in story:
            for phrase in sentence:
                length = len(phrase)
                length_list.append(length)

        length_array = np.array(length_list)
        mean = length_array.mean()
        phrase_lengths.append(float(mean))

    return phrase_lengths


mean_np_length = []
mean_np_length = phrase_length(noun_phrases_corpus, mean_np_length)

mean_vp_length = []
mean_vp_length = phrase_length(verb_phrases_corpus, mean_vp_length)

feature_set['mean_np_length'] = mean_np_length
feature_set['mean_vp_length'] = mean_vp_length

In [ ]:
display(feature_set)

,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,0.550000,0.433333,152.0,79.0,49.0,11.0,13.0,2.354430,4.632653
1,0.650000,0.638095,196.0,93.0,68.0,12.0,23.0,2.516129,7.161765
2,0.148148,0.096959,599.0,296.0,183.0,41.0,79.0,3.094595,7.650273
3,0.152941,0.105062,546.0,285.0,172.0,17.0,72.0,3.010526,6.168605
4,0.600000,0.545455,123.0,64.0,40.0,11.0,8.0,2.484375,6.775000
...,...,...,...,...,...,...,...,...,...
1534,0.461538,0.384615,153.0,70.0,55.0,14.0,14.0,2.871429,9.181818
1535,0.600000,0.400000,64.0,27.0,27.0,5.0,5.0,1.703704,6.481481
1536,0.166667,0.285714,70.0,33.0,20.0,5.0,12.0,2.424242,7.450000
1537,0.500000,0.600000,97.0,49.0,36.0,3.0,9.0,2.204082,5.694444


In [ ]:
feature_set.to_csv('processed_data/syntactic_features.csv')